### P3: Limpando os dados do OpenStreetMap

Daniel Senna Panizzo - Udacity - Área do mapa: Brasília, DF, Brasil 

#### Resumo

O presente projeto tem o intuito de verificar o aprendizado do autor nas lições de limpeza de dados do Nanodegree em Análise de Dados do Udacity. Utilizando os [dados da área de Brasília, DF, Brasil](https://mapzen.com/data/metro-extracts/metro/brasilia_brazil/) do OpenStreetMap, passaremos pelo processo de auditoria, carga e análise dos dados.

#### Fase 01: Aquisição dos dados

Para a execução dos testes deste projeto foi utilizada apenas o área do bairro "Asa Norte" em Brasília, escolha feita por ser meu atual local de moradia. A extração desta amostra de dados foi realizada pela ferramenta de seleção manual do OpenStreetMap utilizando os seguintes parâmetros de latitude e longitude: -15.7300, -47.8480, -15.7940, -47.9080 (inseridos no sentido horário).

Abaixo exploraremos quais elementos XML estão disponíveis na nossa extração de dados.

In [1]:
##################################
#       BIBLIOTECAS PYTHON       #
##################################
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
from collections import defaultdict
import cerberus
import schema
import json

# Caminho do arquivo de amostra de dados do OpenStreetMap
# OSM_PATH = "brasilia_sample.osm"
# Caminho do arquivo completo de dados do OpenStreetMap
OSM_PATH = "brasilia_brazil.osm" 

def count_tags(filename):
    """
    Funcao para identificar e contar a quantidade de tags em um arquivo XML.
    
    Args:
        filename (str): Caminho e nome do arquivo XML.
                        Ex.: "C:\Dados\Arquivo.xml" 
    Returns:
        dic: Key   = Tags identificadas no arquivo XML
             Value = Quantidade de vezes que a tag apareceu no arquivo XML.
             Ex.: {"nome_tag_01": 10, "nome_tag_02": 2}
    """
    tags = {}
    for event, elem in ET.iterparse(filename):
        if elem.tag in tags:
            tags[elem.tag] += 1
        else:
            tags[elem.tag] = 1
    return tags

count_tags(OSM_PATH)

{'bounds': 1,
 'member': 6270,
 'nd': 614950,
 'node': 469773,
 'osm': 1,
 'relation': 897,
 'tag': 217563,
 'way': 94443}

O resultado condiz a descrição do conteúdo de um arquivo OSM, conforme a [Wiki do OpenStreetMaps](https://wiki.openstreetmap.org/wiki/OSM_XML).

ELEMENTO | DESCRIÇÃO
---------|----------
OSM      | Informa a versão da API, a ferramenta que gerou a extração dos dados e informações de licença.
BOUNDS   | Informa as latitudes e longitudes dos pontos máximos e mínimos da área em que os dados estão contidos.
NODE     | Um bloco de *nodes* contendo a localização (latidude e longitude) no sistema de referenciamento WGS84. Cada *node* representa um ponto específico na supercie da Terra. 
WAY      | Um bloco de *ways* referenciando os *nodes* (*nd*) de cada *way*. Um *way* pode ser utilizado para representar elementos lineares como ruas ou rios. Também pode ser utilizado ou para representar os limites de uma área, como um parque ou um prédio. 
RELATION | Um bloco de *relations* referenciando cada um dos *members* de cada *relation*. *Relations* são estruturas de dados de multi-propósito utilizadas para representar relações entre seus elementos (*nodes*, *ways* ou outras *relations*).
TAG      | Todos os tipos de elementos (*nodes*, *ways* e *relations*) podem ter *tags*. *Tags* descrevem o elemento ao qual estão ligados. A descrição é feita no formato chave-valor (*key-value*).



#### Fase 02: Preparação dos dados

Continuando com a análise dos dados que foram inseridos pelos colaboradores do OSM, vamos verificar se no nosso arquivo existe algum elemento *tag* com nome problemático, ou seja, que contenha caracteres incomuns como %, #, $, @, etc.

In [2]:
# Expressoes Regulares (Regular Expressions):
# Para identificar nomes com todos os caracteres minusculos
lower = re.compile(r'^([a-z]|_)*$')

# Para identificar nomes com todos os caracteres minusculos e dois pontos no meio
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')

# Para identificar nomes que contenham qualquer caractere problematico
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    """
    Funcao para classificar o nome da key de um elemento tag entre:
    - lower:       nomes em caixa baixa
    - lower_colon: nomes em caixa baixa com dois pontos no meio
    - problemchar: nomes com caracteres problematicos
    - other:       outro tipo de nome
    
    Args:
        element (cursor): Cursor contendo o elemento XML em analise
    Returns:
        dic: Dicionario contendo a quantidade de nomes em cada classificacao.
    """
    if element.tag == "tag":
        key = element.attrib['k']
        if re.search(lower, key):
            keys['lower'] += 1
        elif re.search(lower_colon, key):
            keys['lower_colon'] += 1
        elif re.search(problemchars, key):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1

    return keys

def process_class_keys(filename):
    """
    Funcao para classificar e contar as keys do elemento tag do arquivo OSM do OpenStreetMap.
    
    Args:
        filename (str): Caminho e nome do arquivo XML.
                        Ex.: "C:\Dados\Arquivo.xml"
    Returns:
        dic: Dicionario contendo a quantidade de nomes em cada classificacao.
             Ex.: {"lower": 100, "lower_colon": 50, "problemchars": 10, "other": 10}
    """
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

process_class_keys(OSM_PATH)

{'lower': 206850, 'lower_colon': 10178, 'other': 535, 'problemchars': 0}

Dentre as *keys* de cada *tag*, nenhum nome foi classificado com caracteres problemáticos. Portanto, podemos seguir com a exploração dos valores dos campos que nos interessam.

O foco do nosso projeto é auditar, principalmente, o endereço fornecido pelos colaboradores do Open Street Maps. A seguir, daremos uma olhada no campo "addr:street" para verficarmos como os usuários estão preenchendo este campo.

In [3]:
def is_key_name(element, key_name):
    """
    Funcao para verificar se a tag e referente ao atributo desejado.
    
    Args:
        element (cursor): Cursor contendo o elemento XML em analise.
        key_name (str): Nome do atributo desejado.
    Returns:
        bool: Verdadeiro se for uma tag do atributo desejado, caso contrario, falso.
    """
    return (element.attrib['k'] == key_name)

def count_key_names(key_names, key_name):
    """
    Funcao para identificar e contar a quantidade de diferentes endereços.
    
    Args:
        key_names (dic): Dicionario contendo a quantidade de cada endereço.
        key_name (str): Nome do endereço verificado.
    """
    if key_name in key_names:
        key_names[key_name] += 1
    else:
        key_names[key_name] = 1

def process_key(filename, key_name):
    """
    Funcao que verifica todas as tags de endereco de um arquivo OSM e lista
    todos os diferentes enderecos e a quantidade de vezes em que aparece no 
    arquivo em um dicionario.
    
    Args:
        filename (str): Caminho e nome do arquivo XML.
                        Ex.: "C:\Dados\Arquivo.xml"   
    Returns:
        dic: Dicionario contendo a quantidade vezes que os endereços aparecem no arquivo.
    """
    osm_file = open(filename, "r")
    street_names = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_key_name(tag, key_name):
                    count_key_names(street_names, tag.attrib['v'])
    osm_file.close()
    return street_names

# Amostra dos dados das tags de endereco
process_key(OSM_PATH, 'addr:street').items()[:10]

[(u'EQ 15/26 \xc1REA COMUNAL 01', 1),
 ('Quadra 14 Conjunto A5', 1),
 (u'Condom\xednio Ch\xe1caras Itaip\xfa, Ch. 58', 2),
 ('SIG Quadra 06', 1),
 (u'QN 09 \xc1rea Central 04', 1),
 (u'QL 32 SEDB - A.E. 01 - lago sul - Bras\xedlia', 1),
 ('CSG 09 Lote 10', 1),
 ('Avenida Dois', 1),
 ('EQS 514-515', 1),
 ('CLN 216 BL. C', 1)]

Como morador de Brasília posso dizer que a maior parte dos endereços me é familiar. No entanto, a grande maioria dos endereços estão abreviados. É comum em Brasília comunicarmos os endereços utilizando apenas as siglas, mas cada uma delas possui um significado.

Podemos encontrar até nomes que tentam conciliar a sigla com o nome completo, como em:

- SCEN Trecho 1 Setor de Clubes Norte

Ainda assim, o nome completo fornecido está errado, já que SCEN é referente a "Setor de Clubes ESPORTIVOS Norte".

Visto que entre moradores da cidade é normal o uso de siglas, ao auditarmos os dados, manteremos a sigla ao final do nome completo do endereço.

A seguir daremos uma olhada no campo "addr:postcode".

In [4]:
# Amostra de dados das tags de codigo postal
process_key(OSM_PATH, 'addr:postcode').items()[:10]

[('70852520', 1),
 ('70680-900', 1),
 ('70342-060', 1),
 ('70236-150', 1),
 ('70236-010', 1),
 ('71570-050', 2),
 ('70640-009', 1),
 ('70775-010', 1),
 ('71060-250', 1),
 ('70.687-230', 1)]

A boa notícia é que os códigos postais (CEPs) parecem estar bem documentados. Assim, podemos aplicar alguns critérios de auditoria de dados. 

Ao verificar a **validade**, podemos observar:
- Se o CEP inicia com o número 7, padrão do Distrito Federal;
- Se o CEP possui 8 digitos numéricos; 

Para verificar a **precisão** e **plenitude** dos CEPs, poderímos utilizar o banco de dados de logradouros dos Correios como referência. Infelizmente este conjunto de dados não é disponibilizado gratuitamente para uso. 

Ao aplicar um tratamento de dados nos CEPs e padronizá-los para terem apenas digitos, podemos testar sua **consistência** ao comparar se diferentes latitudes e longitudes possuem os mesmos CEPs. Podemos, também, considerar que os CEPs estão **uniformes**, visto que não há outro tipo de código postal dentro do território brasileiro.

Como estratégia de tratamento dos dados, como as difereças entre os CEPs residem principalmente na maneira como foram escritas, para padronizá-las iremos retirar todos os outros caracteres que não sejam números. 

Utilizando como base a lista de siglas de Brasília disponível neste [blog](http://siglasbsb.alanmol.com.br/p/siglas.html), seguiremos para o mapeamento das siglas com seus respectivos nomes e padronização da escrita do código postal.

In [5]:
# Expressoes Regulares (Regular Expressions):
# Para identificar a primeira palavra completa do endereço, seguida ou não de ponto
street_abbr_re = re.compile(r'^\b\S+\.?', re.IGNORECASE)
# Para identificar caracteres não numéricos
postcode_clean_re = re.compile(r'\D')

# Lista de siglas esperadas de serem encontradas nos enderecos de Brasilia
expected = ["ADE","AE","AEB","AEMN","AENW","AOS","APO","ARIE","AVPR","BOT","BSB","CA","CADF","CCSW","CEN"
           ,"CES","CE-UNB","CL","CLN","CLRN","CLS","CLSW","CRS","EMI","EMO","EPAA","EPAC","EPAR","EPCA"    
           ,"EPCL","EPCT","EPCV","EPDB","EPGU","EPIA","EPIB","EPIG","EPIP","EPJK","EPNA","EPNB","EPPN"
           ,"EPPR","EPTG","EPTM","EPTT","EPUB","EPVB","EPVL","EPVP","EQN","EQS","ERL","ERN","ERS","ERW"
           ,"ESAF","ETO","ML","PCH","PFB","PFR","PMU","PQEAT","PQEB","PQEN","PQNB","PTP","QELC","QI"      
           ,"QL","QMSW","QRSW","RER-IBGE","SAAN","SAFN","SAFS","SAI/SO","SAIN","SAIS","SAM","SAN","SAUN"
           ,"SAS","SAUS","SBN","SBS","SCEN","SCES","SCIA","SCLRN","SCN","SCS","SCRN","SCRS","SCTN","SCTS","SDC","SDN"
           ,"SDS","SEDB","SEN","SEN","SEPN","SEPS","SES","SEUPS","SFA","SGA","SGAN","SGAS","SGCV","SGMN"
           ,"SGO","SGON","SHA","SHB","SHCES","SHCGN","SHCGS","SHCN","SHCNW","SHCS","SHCSW","SHEP","SHIGS"
           ,"SHIN","SHIP","SHIS","SHLN","SHLS","SHLSW","SHMA","SHN","SHS","SHTN","SHTO","SHTQ","SHTS"
           ,"SIA","SIBS","SIG","SIT","SMA","SMAN","SMAS","SMC","SMDB","SMHN","SMHS","SMIN","SMLN","SMPW"
           ,"SMU","SOF","SPMN","SPO","SPP","SPVP","SQN","SQNW","SQS","SQSW","SRES","SRIA","SRPN","SRPS"  
           ,"SRTVN","SRTVS","STN","STRC","STS","UNB","VPLA","ZC","ZCA","ZE","ZFN","ZI","ZR","ZV"]

# Mapeamento das siglas e seus respectivos nomes completos
mapping = {"ADE" : "Área de Desenvolvimento Econômico (ADE)"
        ,"AE" : "Área Especial (AE)"
        ,"AEB" : "Aeroporto de Brasília (AeB)"
        ,"AEMN" : "Área de Expansão dos Ministérios Norte (AEMN)" 
        ,"AENW" : "Área Especial Noroeste (AENW)"
        ,"AOS" : "Área Octogonal Sul (AOS)" 
        ,"APO" : "Academia de Polícia (APO)"
        ,"ARIE" : "Área de Relevante Interesse Ecológico (ARIE)"
        ,"AVPR" : "Área Verde de Proteção e Reserva (AVPR)"
        ,"BOT" : "Jardim Botânico (BOT)"
        ,"BSB" : "Brasília (BSB)"
        ,"CA" : "Centro de Atividades (CA)"
        ,"CADF" : "Centro Administrativo do Distrito Federal (CADF)" 
        ,"CCSW" : "Centro Comercial Sudoeste (CCSW)"
        ,"CEN" : "Cemitério Norte (CEN)"
        ,"CES" : "Cemitério Sul (CES)"
        ,"CE-UNB" : "Campus Experimental da UnB (CE-UnB)"
        ,"CL" : "Comércio Local (CL)"
        ,"CLN" : "Comércio Local Norte (CLN)"
        ,"CLRN" : "Comércio Local Residencial Norte (CLRN)"
        ,"CLS" : "Comércio Local Sul (CLS)"
        ,"CLSW" : "Comércio Local Sudoeste (CLSW)"
        ,"CRS" : "Comércio Residencial Sul (CRS)"
        ,"EMI" : "Esplanada dos Ministérios (EMI)"
        ,"EMO" : "Eixo Monumental (EMO)"
        ,"EPAA" : "Estrada Parque Armazenamento e Abastecimento (EPAA)"
        ,"EPAC" : "Estrada Parque Acampamento (EPAC)"
        ,"EPAR" : "Estrada Parque Aeroporto (EPAR)"
        ,"EPCA" : "Estrada Parque Centro de Atividades (EPCA)"
        ,"EPCL" : "Estrada Parque Ceilândia (EPCL)"
        ,"EPCT" : "Estrada Parque Contorno (EPCT)"
        ,"EPCV" : "Estrada Parque Cabeça do Veado (EPCV)"
        ,"EPDB" : "Estrada Parque Dom Bosco (EPDB)"
        ,"EPGU" : "Estrada Parque Guará (EPGU)"
        ,"EPIA" : "Estrada Parque Indústria e Abastecimento (EPIA)"
        ,"EPIB" : "Estrada Parque Interbairros (EPIB)"
        ,"EPIG" : "Estrada Parque Indústrias Gráficas (EPIG)"
        ,"EPIP" : "Estrada Parque Ipê (EPIP)"
        ,"EPJK" : "Estrada Parque Juscelino Kubitschek (EPJK)"
        ,"EPNA" : "Estrada Parque das Nações (EPNA)"
        ,"EPNB" : "Estrada Parque Núcleo Bandeirante (EPNB)"
        ,"EPPN" : "Estrada Parque Península Norte (EPPN)"
        ,"EPPR" : "Estrada Parque Paranoá (EPPR)"
        ,"EPTG" : "Estrada Parque Taguatinga (EPTG)"
        ,"EPTM" : "Estrada Parque Tamanduá (EPTM)"
        ,"EPTT" : "Estrada Parque Torto (EPTT)"
        ,"EPUB" : "Estrada Parque Universidade de Brasília (EPUB)"
        ,"EPVB" : "Estrada Parque Vargem Bonita (EPVB)"
        ,"EPVL" : "Estrada Parque Vale (EPVL)"
        ,"EPVP" : "Estrada Parque Vicente Pires (EPVP)"
        ,"EQN" : "Entrequadra Norte (EQN)"
        ,"EQS" : "Entrequadra Sul (EQS)"
        ,"ERL" : "Eixo Rodoviário Leste (ERL)"
        ,"ERN" : "Eixo Rodoviário Norte (ERN)"
        ,"ERS" : "Eixo Rodoviário Sul (ERS)"
        ,"ERW" : "Eixo Rodoviário Oeste (ERW)"
        ,"ESAF" : "Escola de Administração Fazendária (ESAF)"
        ,"ETO" : "Esplanada da Torre (ETO)"
        ,"ML" : "Mansões do Lago (ML)"
        ,"PCH" : "Polígono de Captação Hídrica (PCH)"
        ,"PFB" : "Parque Ferroviário de Brasília (PFB)"
        ,"PFR" : "Plataforma Rodoviária (PFR)"
        ,"PMU" : "Praça Municipal (PMU)"
        ,"PQEAT" : "Parque de Exposição Agropecuária do Torto (PqEAT)"
        ,"PQEB" : "Parque Estação Biológica (PqEB)"
        ,"PQEN" : "Parque Ecológico Norte (PqEN)"
        ,"PQNB" : "Parque Nacional de Brasília (PqNB)"
        ,"PTP" : "Praça dos Três Poderes (PTP)"
        ,"QELC" : "Quadras Econômicas Lúcio Costa (QELC)"
        ,"QI" : "Quadra Interna (QI)"
        ,"QL" : "Quadra do Lago (QL)"
        ,"QMSW" : "Quadra Mista Sudoeste (QMSW)"
        ,"QRSW" : "Quadra Residencial Sudoeste (QRSW)"
        ,"RER-IBGE" : "Reserva Ecológica do Roncador - IBGE (RER-IBGE)"
        ,"SAAN" : "Setor de Armazenagem e Abastecimento Norte (SAAN)"
        ,"SAFN" : "Setor de Administração Federal Norte (SAFN)"
        ,"SAFS" : "Setor de Administração Federal Sul (SAFS)"
        ,"SAI/SO" : "Setor de Áreas Isoladas Sudoeste (SAI/SO)"
        ,"SAIN" : "Setor de Áreas Isoladas Norte (SAIN)"
        ,"SAIS" : "Setor de Áreas Isoladas Sul (SAIS)"
        ,"SAM" : "Setor de Administração Municipal (SAM)"
        ,"SAN" : "Setor de Autarquias Norte (SAN)"
        ,"SAUN" : "Setor de Autarquias Norte (SAUN)"
        ,"SAS" : "Setor de Autarquias Sul (SAS)"
        ,"SAUS" : "Setor de Autarquias Sul (SAUS)"
        ,"SBN" : "Setor Bancário Norte (SBN)"
        ,"SBS" : "Setor Bancário Sul (SBS)"
        ,"SCEN" : "Setor de Clubes Esportivos Norte (SCEN)"
        ,"SCES" : "Setor de Clubes Esportivos Sul (SCES)"
        ,"SCIA" : "Setor Complementar de Indústria e Abastecimento (SCIA)"
        ,"SCLRN" : "Setor Comercial Local Residencial Norte (SCLRN)"
        ,"SCN" : "Setor Comercial Norte (SCN)"
        ,"SCS" : "Setor Comercial Sul (SCS)"
        ,"SCRN": "Setor Comercial Residencial Norte (SCRN)"
        ,"SCRS": "Setor Comercial Residencial Sul (SCRS)"
        ,"SCTN" : "Setor Cultural Norte (SCTN)"
        ,"SCTS" : "Setor Cultural Sul (SCTS)"
        ,"SDC" : "Setor de Divulgação Cultural (SDC)"
        ,"SDN" : "Setor de Diversões Norte (SDN)"
        ,"SDS" : "Setor de Diversões Sul (SDS)"
        ,"SEDB" : "Setor Hermida Dom Bosco (SEDB)"
        ,"SEN" : "Setor de Embaixadas Norte (SEN)"
        ,"SEPN" : "Setor de Edifícios Públicos Norte (SEPN)"
        ,"SEPS" : "Setor de Edifícios Públicos Sul (SEPS)"
        ,"SES" : "Setor de Embaixadas Sul (SES)"
        ,"SEUPS" : "Setor de Edifícios e Utilidades Públicas Sul (SEUPS)"
        ,"SFA" : "Zona Funcional-Administrativa (SFA)"
        ,"SGA" : "Setor de Grandes Áreas (SGA)"
        ,"SGAN" : "Setor de Grandes Áreas Norte (SGAN)"
        ,"SGAS" : "Setor de Grandes Áreas Sul (SGAS)"
        ,"SGCV" : "Setor de Garagens e Concessionárias de Veículos (SGCV)"
        ,"SGMN" : "Setor de Garagens dos Ministérios Norte (SGMN)"
        ,"SGO" : "Setor de Garagens Oficiais (SGO)" 
        ,"SGON" : "Setor de Garagens e Oficinas Norte (SGON)"
        ,"SHA" : "Setor Habitacional Arniqueiras (SHA)"
        ,"SHB" : "Setor Habitacional Buritis (SHB)"
        ,"SHCES" : "Setor de Habitações Coletivas Econômicas Sul (SHCES)"
        ,"SHCGN" : "Setor Habitacional de Casas Geminadas Norte (SHCGN)"
        ,"SHCGS" : "Setor Habitacional de Casas Geminadas Sul (SHCGS)"
        ,"SHCN" : "Setor de Habitações Coletivas Norte (SHCN)"
        ,"SHCNW" : "Setor de Habitações Coletivas Noroeste (SHCNW)"
        ,"SHCS" : "Setor de Habitações Coletivas Sul (SHCS)"
        ,"SHCSW" : "Setor de Habitações Coletivas Sudoeste (SHCSW)"
        ,"SHEP" : "Setor Habitacional Estrada Parque (SHEP)" 
        ,"SHIGS" : "Setor de Habitações Individuais Geminadas Sul (SHIGS)"
        ,"SHIN" : "Setor de Habitações Individuais Norte (SHIN)"
        ,"SHIP" : "Setor Hípico (SHIP)"
        ,"SHIS" : "Setor de Habitações Individuais Sul (SHIS)"
        ,"SHLN" : "Setor Hospitalar Local Norte (SHLN)"
        ,"SHLS" : "Setor Hospitalar Local Sul (SHLS)"
        ,"SHLSW" : "Setor Hospitalar Local Sudoeste (SHLSW)"
        ,"SHMA" : "Setor Habitacional Jardins Mangueiral (SHMA)"
        ,"SHN" : "Setor Hoteleiro Norte (SHN)"
        ,"SHS" : "Setor Hoteleiro Sul (SHS)"
        ,"SHTN" : "Setor de Hotéis e Turismo Norte (SHTN)"
        ,"SHTO" : "Setor Habitacional Tororó (SHTo)"
        ,"SHTQ" : "Setor Habitacional Taquari (SHTQ)"
        ,"SHTS" : "Setor de Hotéis e Turismo Sul (SHTS)"
        ,"SIA" : "Setor de Indústria e Abastecimento (SIA)"
        ,"SIBS" : "Setor de Indústrias Bernardo Sayão (SIBS)"
        ,"SIG" : "Setor de Indústrias Gráficas (SIG)"
        ,"SIT" : "Setor Invernada do Torto (SIT)"
        ,"SMA" : "Setor de Múltiplas Atividades do Gama (SMA)"
        ,"SMAN" : "Setor de Múltiplas Atividades Norte (SMAN)"
        ,"SMAS" : "Setor de Múltiplas Atividades Sul (SMAS)"
        ,"SMC" : "Setor Militar Complementar (SMC)"
        ,"SMDB" : "Setor de Mansões Dom Bosco (SMDB)"
        ,"SMHN" : "Setor Médico Hospitalar Norte (SMHN)"
        ,"SMHS" : "Setor Médico Hospitalar Sul (SMHS)"
        ,"SMIN" : "Setor de Mansões Isoladas Norte (SMIN)"
        ,"SMLN" : "Setor de Mansões Lago Norte (SMLN)"
        ,"SMPW" : "Setor de Mansões Park Way (SMPW)"
        ,"SMU" : "Setor Militar Urbano (SMU)"
        ,"SOF" : "Setor de Oficinas (SOF)"
        ,"SPMN" : "Setor de Postos e Moteis (SPMN)"
        ,"SPO" : "Setor Policial (SPO)"
        ,"SPP" : "Setor Palácio Presidencial (SPP)"
        ,"SPVP" : "Setor de Preservação da Vila Planalto (SPVP)"
        ,"SQN" : "Superquadra Norte (SQN)"
        ,"SQNW" : "Superquadra Noroeste (SQNW)"
        ,"SQS" : "Superquadra Sul (SQS)"
        ,"SQSW" : "Superquadra Sudoeste (SQSW)"
        ,"SRES" : "Setor de Residências Econômicas Sul (SRES)"
        ,"SRIA" : "Setor Residencial Indústria e Abastecimento (SRIA)"
        ,"SRPN" : "Setor de Recreação Pública Norte (SRPN)"
        ,"SRPS" : "Setor de Recreação Pública Sul (SRPS)"
        ,"SRTVN" : "Setor de Rádio e Televisão Norte (SRTVN)"
        ,"SRTVS" : "Setor de Rádio e Televisão Sul (SRTVS)"
        ,"STN" : "Setor Terminal Norte (STN)"
        ,"STRC" : "Setor de Transporte Regional de Cargas (STRC)"
        ,"STS" : "Setor Terminal Sul (STS)"
        ,"UNB" : "Universidade de Brasília (UnB)"
        ,"VPLA" : "Vila Planalto (VPLA)"
        ,"ZC" : "Zona Central (ZC)"
        ,"ZCA" : "Zona Cívico-Administrativa (ZCA)"
        ,"ZE" : "Zona Especial (ZE)"
        ,"ZFN" : "Zona Industrial (ZfN)"
        ,"ZI" : "Zona Institucional (ZI)"
        ,"ZR" : "Zona Residencial (ZR)"
        ,"ZV" : "Zona Verde (ZV)"}

def update_name(name, mapping):
    """
    Funcao que verifica se existe uma abreviacao conhecida no endereco e a substitui pelo
    nome completo.
    
    Args:
        name (str): Nome do endereco a ser verificado.
        mapping (dic): Dicionario contendo a relacao de abreviacoes e seus respectivos nomes completos.
    Returns:
        str: Nome do endereco com a abreviacao substituida pelo nome completo.
    """
    m = street_abbr_re.search(name)
    if m:
        street_abbr = m.group()
        if street_abbr.upper() in expected:
            name = re.sub(street_abbr_re
                         ,mapping[street_abbr.upper()]
                         ,name.encode('utf-8'))
    return name

def update_postcode(postcode):
    """
    Funcao que limpa o codigo postal de caracteres não numéricos.
    
    Args:
        postcode (str): Codigo postal a ser verificado.
    Returns:
        str: Codigo postal apenas com numeros.
    """
    m = postcode_clean_re.search(postcode)
    if m:
        postcode_clean = m.group()
        postcode = re.sub(postcode_clean_re, '', postcode)
    return postcode

In [6]:
# Teste do tratamento do endereço
update_name('Scen Trecho 2', mapping)

'Setor de Clubes Esportivos Norte (SCEN) Trecho 2'

In [7]:
# Teste do tratamento do codigo postal
update_postcode('70.767-0 1 0')

'70767010'

Ok, com as funções de tratamento dos campos de endereço prontas, podemos preparar os dados para serem inseridos no banco de dados.

In [8]:
# Caminhos e nomes dos arquivos CSV 
# gerados após o tratamento dos dados
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

# Modelo dos dados 
SCHEMA = schema.schema

# Ordem das colunas do modelo SQL 
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def update_value(key, value):
    """
    Funcao que verifica se o campo e referente a um endereco ou codigo postal
    e aplica o respectivo tratamento de dados quando necessario.
    
    Args:
        key (str): Nome do campo em analise.
        value (str): Valor do campo em analise.
    Returns:
        str: Valor do campo tratado em caso de endereco ou codigo postal.
    """
    if key == 'street':
        return update_name(value, mapping)
    elif key == 'postcode':
        return update_postcode(value)
    else:
        return value

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=problemchars, default_tag_type='regular'):
    """
    Funcao para limpar e modelar os elementos node ou way do XML para um dicionario Python.
    
    Args:
        element (cursor): Cursor contendo o elemento XML (tags node ou way) em analise.
        node_attr_fields (array): Lista de campos de interesse da tag node.
        way_attr_fields (array): Lista de campos de interesse da tag way.
        problem_chars (regex): Funcao regex para identificacao de caracteres invalidos.
        default_tag_type (str): Nome dado ao tipo de dado padrao.
    Returns:
        dic: Dicionario contendo a tag node ou way modelada e tratada.
    """

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    # Se for uma tag NODE
    if element.tag == 'node':

        # Para cada atributo da tag NODE
        for attr in element.attrib:
            # Insere o valor do atributo da tag se o atributo
            # estiver listado dentre as colunas do modelo SQL
            if attr in node_attr_fields:
                node_attribs[attr] = element.attrib[attr]

        # Para cada subtag da tag NODE
        for sub in element:
            # Armazena o nome do atributo "k" (key) da subtag
            key = sub.get("k")
            # Divide o nome da key da subtag em duas se houver dois pontos (:),
            # ou seja, divide o nome da key se for uma subtag de endereco (addr)
            keys = re.split(':',key,1)

            # Se nao houver problemas no nome da key
            if problem_chars.search(key) == None:
                tag_dic = {}
                # Armazena o id da subtag
                tag_dic['id'] = node_attribs['id']
                # Armazena a key da subtag, se for uma key de endereco insere e segunda parte da key
                tag_dic['key'] = keys[0] if len(keys) == 1 else keys[1]
                # Armazena o value da subtag, se for um value de endereco passa pelo tratamento dos dados
                tag_dic['value'] = sub.get("v") if len(keys) == 1 else update_value(keys[1], sub.get("v"))
                # Armazena o type da subtag, pode ser uma subtag regular ou subtag de endereco (addr)
                tag_dic['type'] = default_tag_type if len(keys) == 1 else keys[0]

                # Insere os attributos da subtag no dicionario
                tags.append(tag_dic)

        return {'node': node_attribs, 'node_tags': tags}

    # Se for uma tag WAY
    elif element.tag == 'way':

        # Para cada atributo da tag WAY
        for attr in element.attrib:
            # Insere o valor do atributo da tag se o atributo
            # estiver listado dentre as colunas do modelo SQL
            if attr in way_attr_fields:
                way_attribs[attr] = element.attrib[attr]

        # Para cada subtag da tag WAY
        for idx, sub in enumerate(element):

            # Se for uma subtag TAG
            if sub.tag == 'tag':
                # Armazena o nome do atributo "k" (key) da subtag
                key = sub.get("k")
                # Divide o nome da key da subtag em duas se houver dois pontos (:),
                # ou seja, divide o nome da key se for uma subtag de endereco (addr)
                keys = re.split(':',key,1)

                if problem_chars.search(key) == None:
                    tag_dic = {}
                    # Armazena o id da subtag
                    tag_dic['id'] = way_attribs['id']
                    # Armazena a key da subtag, se for uma key de endereco insere e segunda parte da key
                    tag_dic['key'] = keys[0] if len(keys) == 1 else keys[1]
                    # Armazena o value da subtag, se for um value de endereco passa pelo tratamento dos dados
                    tag_dic['value'] = sub.get("v") if len(keys) == 1 else update_value(keys[1], sub.get("v"))
                    # Armazena o type da subtag, pode ser uma subtag regular ou subtag de endereco (addr)
                    tag_dic['type'] = default_tag_type if len(keys) == 1 else keys[0]

                    # Insere os attributos da subtag no dicionario
                    tags.append(tag_dic)

            # Se for uma subtag NODE
            if sub.tag == 'nd':
                tag_dic = {}
                # Armazena o id da subtag
                tag_dic['id'] = way_attribs['id']
                # Armazena o id do NODE referenciado 
                tag_dic['node_id'] = sub.get("ref")
                # Armazena a posicao da subtag
                tag_dic['position'] = idx

                # Insere os attributos da subtag no dicionario
                way_nodes.append(tag_dic)

        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

#========================================================#
# ABAIXO ESTAO AS FUNCOES FORNECIDAS PELO ESTUDO DE CASO #
#     PARA AUXILIO NA ESCRITA DOS DADOS PARA CSV         #
#========================================================#

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)

        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, fieldnames=NODE_FIELDS, delimiter='|')
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, fieldnames=NODE_TAGS_FIELDS, delimiter='|')
        ways_writer = UnicodeDictWriter(ways_file, fieldnames=WAY_FIELDS, delimiter='|')
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, fieldnames=WAY_NODES_FIELDS, delimiter='|')
        way_tags_writer = UnicodeDictWriter(way_tags_file, fieldnames=WAY_TAGS_FIELDS, delimiter='|')

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

In [9]:
process_map(OSM_PATH, validate=True);

Após a execução do código, os dados em XML do OpenStreetMap foram formatados em CSV nos seguintes arquivos:

Em seguida modificaremos nossas funções para formatar os dados XML em JSON.

In [10]:
# Lista dos campos de interesse sobre a criacao do elemento
CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

def shape_element(element):
    """
    Funcao para limpar e modelar os elementos node ou way do XML para um dicionario Python.
    
    Args:
        element (cursor): Cursor contendo o elemento XML (tags node ou way) em analise.
    Returns:
        dic: Dicionario contendo a tag node ou way modelada e tratada.
    """
    node = {}
    
    # Se for uma tag NODE ou WAY
    if element.tag == "node" or element.tag == "way" :
        
        # Prepara as variaveis para insercao no documento (JSON)
        node["type"] = element.tag
        node['created'] = {}
        pos = []
        node_refs = []
        address = {}


        # Para cada atributo na tag
        for attr in element.attrib:
            # Se o atributo estiver na lista de interesse dos campos de criacao
            if attr in CREATED:
                # Armazena o atributo dentro do dicionario 'created'
                node['created'][attr] = element.attrib[attr]
            # Se for um atributo de latidude ou longitude
            elif attr in ["lat","lon"]:
                # Armazena o atributo dentro do array de posicao 'pos'
                pos.append(float(element.attrib[attr]))
            # Para os outros atributos
            else:
                # Armazena o valor e o nome do atributo
                node[attr] = element.attrib[attr]

        # Para cada subtag na tag NODE ou WAY
        for sub in element:
            # Se for uma subtag TAG
            if sub.tag == 'tag':
                # Armazena o nome do atributo "k" (key) da subtag
                key = sub.get("k")
                # Divide o nome da key da subtag em duas se houver dois pontos (:),
                # ou seja, divide o nome da key se for uma subtag de endereco (addr)
                keys = re.split(':',key)

                # Se nao houver problemas no nome da key
                if problemchars.search(key) == None:
                    # Se for um campo de endereco (addr)
                    if keys[0] == 'addr' and len(keys) <= 2:
                        # Armazena o endereco tratado no dicionario 'adress'
                        address[keys[1]] = update_value(keys[1],sub.get("v"))
                    # Se for uma key com dois valores
                    elif len(keys) == 2:
                        # Cria um dicionario com o nome do primeiro valor
                        node[keys[0]] = {}
                        # Armazena o valor dentro do dicionario criado
                        node[keys[0]][keys[1]] = sub.get("v")
                    # Para keys com apenas um valor
                    else:
                        # Armazena a chave e valor
                        node[keys[0]] = sub.get("v")

            # Se for uma subtag NODE (nd)
            if sub.tag == 'nd':
                # Amazena a referencia ao node no array 'node_refs'
                node_refs.append(sub.get("ref"))

        # Inclui os arrays no dicionario apenas se tiverem valores
        if bool(node_refs):
            node['node_refs'] = node_refs
        if bool(address):
            node['address'] = address
        if bool(pos):
            node['pos'] = pos

        return node
    else:
        return None

#========================================================#
# ABAIXO ESTAO AS FUNCOES FORNECIDAS PELO ESTUDO DE CASO #
#     PARA AUXILIO NA ESCRITA DOS DADOS PARA JSON        #
#========================================================#
def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [11]:
process_map(OSM_PATH, True);

Após a execução do código, os dados em XML do OpenStreetMap foram formatados em JSON no seguinte arquivo:

Com os dados tratados, podemos inserí-los nos bancos de dados. Os dados em CSV serão inseridos no seguinte modelo de dados SQL:

Para importar nossos arquivos CSV para suas respectivas tabelas, é necessário executar as seguintes instruções no *shell* do SQLite:

Os dados em JSON seguem o seguinte modelo de documento:

Para importar nosso arquivo JSON para uma coleção no MongoDB, fui utilizada a seguinte instrução no *shell* do MongoDB:

#### Fase 03: Exploração dos dados

Com os dados carregados nos bancos de dados, podemos iniciar a exploração.

**NÚMERO DE *NODES* **

**NÚMERO DE *WAYS* **

**NÚMERO DE USUÁRIOS ÚNICOS**

** TOP 10 CONTRIBUIDORES **

**TOP 10 TIPOS DE LOCAIS**

**PRINCIPAL TIPO DE RESTAURANTE**


**QUADRA COM MAIOR QUANTIDADE DE BARES**

**PIZZARIAS NAS PROXIMIDADES DA MINHA QUADRA**

#### IDEIAS ADICIONAIS

O projeto do Open Street Maps é, por si só, uma iniciativa sensacional, mas basta dar uma olhada nos dados e nos blogs/diários dos editores para entender o quão difícil é manter dados de qualidade quando qualquer um pode editá-los. Talvez a possibilidade de criar e ingressar em grupos de interesse do Open Street Maps possa facilitar a comunicação de usuários e/ou entidades que querem editar um local em comum.   

#### CONCLUSÃO

Após a exploração e análise dos dados, fica claro que o mapa da área de Brasília está incompleto. A documentação, apesar de sugerir padrões, parece ser pouco utilizada pelos colaboradores. O pouco uso destes padrões acaba dificultando a auditoria dos dados. Como sugestão, o OpenStreetMap poderia investir na criação de uma interface mais amigável que guie os colaboradores no momento da inserção dos dados, ajudando a melhorar a qualidade dos dados fornecidos. 

#### REFERÊNCIAS

- https://docs.google.com/document/d/1F0Vs14oNEs2idFJR3C_OPxwS6L0HPliOii-QpbmrMo4/pub 
- https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md

- https://wiki.openstreetmap.org/wiki/OSM_XML
- https://wiki.openstreetmap.org/wiki/Elements

- https://docs.mongodb.com/manual/core/2dsphere/
- https://docs.mongodb.com/manual/reference/operator/query/near/#op._S_near
- https://docs.mongodb.com/manual/tutorial/geospatial-tutorial/

- http://regexr.com/
- https://www.codeschool.com/courses/breaking-the-ice-with-regular-expressions

- https://mapzen.com/data/metro-extracts/metro/brasilia_brazil/
- http://siglasbsb.alanmol.com.br/p/siglas.html

- http://www.tutorialspoint.com/sqlite/index.htm 
- https://sqlite.org/lang.html 
- https://stackoverflow.com/questions/29577713/string-field-value-length-in-mongodb/29578020
- https://stackoverflow.com/questions/18501064/mongodb-aggregation-counting-distinct-fields
